In [4]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
def load_pdf(file):
    from langchain.document_loaders import PyPDFLoader
    print(f'Loading {file}')
    loader = PyPDFLoader(file)
    data = loader.load()
    return data

In [9]:
# Testing the load_pdf function on test.pdf
data = load_pdf('./test.pdf')

#print(data[5].page_content)
#print(data[10].metadata)
#print(f'There are {len(data)} pages in your PDF')

Loading ./test.pdf
There are 259 pages in your PDF


In [10]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks